## Building the Document Index

The purpose of this notebook is to access and prepare the ICBF data for use with the QA Bot accelerator. This worked is based on the Databricks Industry Solutions Accelerator available at https://github.com/databricks-industry-solutions/diy-llm-qa-bot.

##Introduction

So that our qabot application can respond to user questions with relevant answers, we will provide our model with content from documents relevant to the question being asked.  The idea is that the bot will leverage the information in these documents as it formulates a response.

For our application, we've previously extracted a series of documents from the [Colombian Institute for Family Welfare](https://www.icbf.gov.co/servicios/preguntas-y-respuestas-frecuentes) (ICBF, its acronym in Spanish) using web scraping techniques. These documents contain common questions and answers asked by Colombian and foreign citizens regarding children's welfare and several legal procedures related to the topic. Using this documentation source, we will provide context that will allow this bot to respond to questions relevant to this subject area with deep expertise.

In this notebook, we will load these documents stored in a CSV file to a table in the Databricks environment.  We will retrieve those documents along with metadata about them and feed that to a vector store which will create on index enabling fast document search and retrieval.

In [0]:
%pip install langchain==0.0.250 tiktoken==0.4.0 openai==0.27.6 faiss-cpu==1.7.4 typing-inspect==0.8.0 typing_extensions==4.5.0

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f08eb883-ffc6-493d-a8ab-c7cd0d0718ae
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Not uninstalling tenacity at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f08eb883-ffc6-493d-a8ab-c7cd0d0718ae
    Can't uninstall 'tenacity'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import pyspark.sql.functions as fn

import json

from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

In [0]:
%run "./utils/config_utils"

##Step 1: Load the Raw Data to Table

A snapshot of the three documentation sources is made available at a publicly accessible cloud storage. Our first step is to access the extracted documents. We can load them to a table using a Spark DataReader configured for reading [JSON](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrameReader.json.html) with the `multiLine` option.

In [0]:
raw = (
  spark
    .read
    .format("csv")
    .option("inferSchema","true")
    .option("header","true")
    .option("sep", "|")
    .load(
      f"{config['kb_documents_path']}"
      )
  )

display(raw)

id,title,answer,url
PSP01013,"¿El otro progenitor de mi hijo no desea darle el permiso de salida del país, qué debo hacer?","En este caso, usted debe presentar una demanda ante un Juez de Familia, a fin de que este resuelva la controversia, mediante sentencia.",https://www.icbf.gov.co/el-otro-progenitor-de-mi-hijo-no-desea-darle-el-permiso-de-salida-del-pais-que-debo-hacer
CUS01001,¿Qué es la Custodia?,"Es la tenencia física, el cuidado personal y directo del niño, niña o adolescente. Es un derecho de los niños y una obligación de los padres o representantes legales.",https://www.icbf.gov.co/que-es-la-custodia
CUS01002,¿Cómo hago para ver a mi hijo(a) si no tengo la custodia?,"Usted puede solicitar la fijación o regulación de visitas en el Centro Zonal de Bienestar Familiar del lugar en donde se encuentre el niño, niña o adolescente, ya que por Ley es la autoridad competente para conocer y decidir sobre este asunto. Allí, deberá presentarse el registro civil de nacimiento del niño(a), copia de la cedula del solicitante, pruebas de los hechos fundamento de las pretensiones, entre otras. Las visitas se regulan de acuerdo con las necesidades del hijo(a) y a la capacidad del padre o madre de brindarle la atención, cuidado y protección requerida durante el término que duren. Sin embargo, el padre que resida con su menor hijo, también podrá acudir al Juez de Familia para que reglamente las visitas, previo agotamiento del requisito de procedibilidad, esto es, previa conciliación extrajudicial.",https://www.icbf.gov.co/como-hago-para-ver-mi-hijoa-si-no-tengo-la-custodia
CUS01003,¿Cuáles son los causales que pueden dar lugar a la fijación de la custodia?,"No existen causales establecidas en la ley para este hecho, sin embargo corresponde a las autoridades competentes tanto administrativas como judiciales, analizar los hechos de cada caso en concreto a la luz de la normatividad vigente sobre la materia y adoptar una decisión pensando siempre en el bienestar del niño o niña. Los casos más frecuentes son: cuando los padres del menor de edad no conviven juntos y por ello se debe otorgar la tenencia a uno de ellos; también cuando alguno de los padres, o el cuidador que ostente la custodia no sean garantes de sus derechos. También si existe acuerdo entre los padres y los familiares en cuanto a la custodia, pueden acudir al Centro Zonal del lugar donde el niño viva, a fin de que se verifique la viabilidad por parte del Defensor de Familia o autoridad competente.",https://www.icbf.gov.co/cuales-son-los-causales-que-pueden-dar-lugar-la-fijacion-de-la-custodia
CUS01004,¿Dónde puedo realizar una diligencia de fijación de custodia?,"En el Centro Zonal del ICBF más cercano al lugar de residencia del niño, niña o adolescente.Comisaría de Familia más cercano al lugar de residencia del niño, niña o adolescente.Notarias.Centros de Conciliación de Universidades o Privados, legalmente autorizados por la ley.Delegados regionales y seccionales de la Defensoría del Pueblo.Agentes del Ministerio PúblicoJueces de Familia. A falta de todos los anteriores, en el respectivo municipio, esta conciliación podrá ser adelantada por los Personeros y los Jueces Civiles y Promiscuos Municipales. Cabe aclarar que en los lugares del territorio colombiano en donde el Bienestar Familiar no preste sus servicios, así no exista violencia intrafamiliar, podrán acudir a la Comisaria de Familia o Inspección de Policía más cercana al lugar de residencia del niño, niña o adolescente.",https://www.icbf.gov.co/donde-puedo-realizar-una-diligencia-de-fijacion-de-custodia
CUS01005,¿Es válido que los padres de común acuerdo definan quien tendrá la custodia sin necesidad de acudir a alguna entidad?,"Sí, es completamente valido que los padres definan la custodia de forma privada sin acudir a ninguna entidad; sin embargo lo recomendable es que lo hagan por escrito ante una autoridad competente. Es preciso recordar que cualquier decisión que se tome respecto a los hijos debe propender por su

We can persist this data to a table as follows:

In [0]:
# save data to table
_ = (
  raw
    .write
    .format('delta')
    .mode('overwrite')
    .option('overwriteSchema','true')
    .saveAsTable('sources')
  )

# count rows in table
print(spark.table('sources').count())

339


##Step 2: Prepare Data for Indexing

While there are many fields avaiable to us in our newly loaded table, the fields that are relevant for our application are:

* answer - Documentation text or knowledge base response which may include relevant information about user's question
* url - the url pointing to the online document

In [0]:
raw_inputs = (
  spark
    .table('sources')
    .selectExpr(
      'answer',
      'url'
      )
  ) 

display(raw_inputs)

answer,url
"En este caso, usted debe presentar una demanda ante un Juez de Familia, a fin de que este resuelva la controversia, mediante sentencia.",https://www.icbf.gov.co/el-otro-progenitor-de-mi-hijo-no-desea-darle-el-permiso-de-salida-del-pais-que-debo-hacer
"Es la tenencia física, el cuidado personal y directo del niño, niña o adolescente. Es un derecho de los niños y una obligación de los padres o representantes legales.",https://www.icbf.gov.co/que-es-la-custodia
"Usted puede solicitar la fijación o regulación de visitas en el Centro Zonal de Bienestar Familiar del lugar en donde se encuentre el niño, niña o adolescente, ya que por Ley es la autoridad competente para conocer y decidir sobre este asunto. Allí, deberá presentarse el registro civil de nacimiento del niño(a), copia de la cedula del solicitante, pruebas de los hechos fundamento de las pretensiones, entre otras. Las visitas se regulan de acuerdo con las necesidades del hijo(a) y a la capacidad del padre o madre de brindarle la atención, cuidado y protección requerida durante el término que duren. Sin embargo, el padre que resida con su menor hijo, también podrá acudir al Juez de Familia para que reglamente las visitas, previo agotamiento del requisito de procedibilidad, esto es, previa conciliación extrajudicial.",https://www.icbf.gov.co/como-hago-para-ver-mi-hijoa-si-no-tengo-la-custodia
"No existen causales establecidas en la ley para este hecho, sin embargo corresponde a las autoridades competentes tanto administrativas como judiciales, analizar los hechos de cada caso en concreto a la luz de la normatividad vigente sobre la materia y adoptar una decisión pensando siempre en el bienestar del niño o niña. Los casos más frecuentes son: cuando los padres del menor de edad no conviven juntos y por ello se debe otorgar la tenencia a uno de ellos; también cuando alguno de los padres, o el cuidador que ostente la custodia no sean garantes de sus derechos. También si existe acuerdo entre los padres y los familiares en cuanto a la custodia, pueden acudir al Centro Zonal del lugar donde el niño viva, a fin de que se verifique la viabilidad por parte del Defensor de Familia o autoridad competente.",https://www.icbf.gov.co/cuales-son-los-causales-que-pueden-dar-lugar-la-fijacion-de-la-custodia
"En el Centro Zonal del ICBF más cercano al lugar de residencia del niño, niña o adolescente.Comisaría de Familia más cercano al lugar de residencia del niño, niña o adolescente.Notarias.Centros de Conciliación de Universidades o Privados, legalmente autorizados por la ley.Delegados regionales y seccionales de la Defensoría del Pueblo.Agentes del Ministerio PúblicoJueces de Familia. A falta de todos los anteriores, en el respectivo municipio, esta conciliación podrá ser adelantada por los Personeros y los Jueces Civiles y Promiscuos Municipales. Cabe aclarar que en los lugares del territorio colombiano en donde el Bienestar Familiar no preste sus servicios, así no exista violencia intrafamiliar, podrán acudir a la Comisaria de Familia o Inspección de Policía más cercana al lugar de residencia del niño, niña o adolescente.",https://www.icbf.gov.co/donde-puedo-realizar-una-diligencia-de-fijacion-de-custodia
"Sí, es completamente valido que los padres definan la custodia de forma privada sin acudir a ninguna entidad; sin embargo lo recomendable es que lo hagan por escrito ante una autoridad competente. Es preciso recordar que cualquier decisión que se tome respecto a los hijos debe propender por su bienestar.",https://www.icbf.gov.co/es-valido-que-los-padres-de-comun-acuerdo-definan-quien-tendra-la-custodia-sin-necesidad-de-acudir
"No, en Colombia no existe una norma expresa sobre la denominada custodia compartida.",https://www.icbf.gov.co/en-colombia-esta-regulada-la-custodia-compartida
"No se requiere el permiso del padre o madre a quien se le privó la Patria Potestad, sólo se debe presentar copia de la sentencia Judicial que ordena la privación, debidamente ejecutoriada.",https:

The content available within each doc varies but some documents can be quite long.  Here is an example of a large document in our dataset:

In [0]:
long_text = (
  raw_inputs
    .select('answer') # get just the text field
    .orderBy(fn.expr("len(answer)"), ascending=False) # sort by length
    .limit(1) # get top 1
     .collect()[0]['answer'] # pull text to a variable
  )

# display long_text
print(long_text)

Hay tres situaciones por las cuales un niño, niña o adolescente ingresa a un Proceso Administrativo de Restablecimiento de Derechos: Inobservancia: Se presenta cuanto una entidad (administrativa o judicial) nacional o extranjera, o cuando una persona natural, incumple el deber de garantizar los derechos de los niños, niñas o adolescentes, en territorio colombiano o fuera de él. Amenaza: Consiste en toda situación que pone en inminente peligro o riesgo los derechos de los niños, las niñas o adolescentes. Vulneración: Es toda situación de daño, lesión o perjuicio que impide el ejercicio pleno de los derechos de los niños, niñas y adolescentes. Cuando se sospeche que la inobservancia, vulneración o amenaza se da sobre un niño, niña o adolescente que pertenece a una comunidad indígena, las entidades deberán coordinar y articular con la Autoridad Tradicional, todas las acciones de protección, respetando siempre la jurisdicción indígena. Existen unos casos en donde el Defensor de Familia, Co

The process of converting a document to an index involves us translating it to a fixed-size embedding.  An embedding is a set of numerical values, kind of like a coordinate, that summarizes the content in a unit of text. While large embeddings are capable of capturing quite a bit of detail about a document, the larger the document submitted to it, the more the embedding generalizes the content.  It's kind of like asking someone to summarize a paragraph, a chapter or an entire book into a fixed number of dimensions.  The greater the scope, the more the summary must eliminate detail and focus on the higher-level concepts in the text.

A common strategy for dealing with this when generating embeddings is to divide the text into chunks.  These chunks need to be large enough to capture meaningful detail but not so large that key elements get washed out in the generalization.  Its more of an art than a science to determine an appropriate chunk size, but here we'll use a very small chunk size to illustrate what's happening in this step:

In [0]:
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)
for chunk in text_splitter.split_text(long_text):
  print(chunk, '\n')

Hay tres situaciones por las cuales un niño, niña o adolescente ingresa a un Proceso Administrativo de Restablecimiento de Derechos: Inobservancia: Se presenta cuanto una entidad (administrativa o judicial) nacional o extranjera, o cuando una persona natural, incumple el deber de garantizar los derechos de los ni 

arantizar los derechos de los niños, niñas o adolescentes, en territorio colombiano o fuera de él. Amenaza: Consiste en toda situación que pone en inminente peligro o riesgo los derechos de los niños, las niñas o adolescentes. Vulneración: Es toda situación de daño, lesión o perjuicio que imp 

, lesión o perjuicio que impide el ejercicio pleno de los derechos de los niños, niñas y adolescentes. Cuando se sospeche que la inobservancia, vulneración o amenaza se da sobre un niño, niña o adolescente que pertenece a una comunidad indígena, las entidades deberán coordinar y articular con la 

 deberán coordinar y articular con la Autoridad Tradicional, todas las acciones de prote

Please note that we are specifying overlap between our chunks.  This is to help avoid the arbitrary separation of words that might capture a key concept. 

We have set our overlap size very small for this demonstration but you may notice that overlap size does not neatly translate into the exact number of words that will overlap between chunks. This is because we are not splitting the content directly on words but instead on byte-pair encoding tokens derived from the words that make up the text.  You can learn more about byte-pair encoding [here](https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt) but just note that its a frequently employed mechanism for compressing text in many LLM algorithms.

With the concept of document splitting under our belt, let's write a function to divide our documents into chunks and apply it to our data. Note that we are setting the chunk size and overlap to higher values for this step to better align with the [limits](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them) specified with the Chat-GPT model we will eventually transmit this information to.  You might be able to set these values higher but please note that a fixed number of *tokens* are currently allowed with each Chat-GPT model request and that the entire user prompt (including context) and the generated response must fit within that token limit.  Otherwise, an error will be generated:

In [0]:
chunk_size = 3500
chunk_overlap = 400

In [0]:
@fn.udf('array<string>')
def get_chunks(text):

  # instantiate tokenization utilities
  text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  
  # split text into chunks
  return text_splitter.split_text(text)


# split text into chunks
chunked_inputs = (
  raw_inputs
    .withColumn('chunks', get_chunks('answer')) # divide text into chunks
    .drop('answer')
    .withColumn('num_chunks', fn.expr("size(chunks)"))
    .withColumn('chunk', fn.expr("explode(chunks)"))
    .drop('chunks')
    .withColumnRenamed('chunk','text')
  )

  # display transformed data
display(chunked_inputs)

url,num_chunks,text
https://www.icbf.gov.co/el-otro-progenitor-de-mi-hijo-no-desea-darle-el-permiso-de-salida-del-pais-que-debo-hacer,1,"En este caso, usted debe presentar una demanda ante un Juez de Familia, a fin de que este resuelva la controversia, mediante sentencia."
https://www.icbf.gov.co/que-es-la-custodia,1,"Es la tenencia física, el cuidado personal y directo del niño, niña o adolescente. Es un derecho de los niños y una obligación de los padres o representantes legales."
https://www.icbf.gov.co/como-hago-para-ver-mi-hijoa-si-no-tengo-la-custodia,1,"Usted puede solicitar la fijación o regulación de visitas en el Centro Zonal de Bienestar Familiar del lugar en donde se encuentre el niño, niña o adolescente, ya que por Ley es la autoridad competente para conocer y decidir sobre este asunto. Allí, deberá presentarse el registro civil de nacimiento del niño(a), copia de la cedula del solicitante, pruebas de los hechos fundamento de las pretensiones, entre otras. Las visitas se regulan de acuerdo con las necesidades del hijo(a) y a la capacidad del padre o madre de brindarle la atención, cuidado y protección requerida durante el término que duren. Sin embargo, el padre que resida con su menor hijo, también podrá acudir al Juez de Familia para que reglamente las visitas, previo agotamiento del requisito de procedibilidad, esto es, previa conciliación extrajudicial."
https://www.icbf.gov.co/cuales-son-los-causales-que-pueden-dar-lugar-la-fijacion-de-la-custodia,1,"No existen causales establecidas en la ley para este hecho, sin embargo corresponde a las autoridades competentes tanto administrativas como judiciales, analizar los hechos de cada caso en concreto a la luz de la normatividad vigente sobre la materia y adoptar una decisión pensando siempre en el bienestar del niño o niña. Los casos más frecuentes son: cuando los padres del menor de edad no conviven juntos y por ello se debe otorgar la tenencia a uno de ellos; también cuando alguno de los padres, o el cuidador que ostente la custodia no sean garantes de sus derechos. También si existe acuerdo entre los padres y los familiares en cuanto a la custodia, pueden acudir al Centro Zonal del lugar donde el niño viva, a fin de que se verifique la viabilidad por parte del Defensor de Familia o autoridad competente."
https://www.icbf.gov.co/donde-puedo-realizar-una-diligencia-de-fijacion-de-custodia,1,"En el Centro Zonal del ICBF más cercano al lugar de residencia del niño, niña o adolescente.Comisaría de Familia más cercano al lugar de residencia del niño, niña o adolescente.Notarias.Centros de Conciliación de Universidades o Privados, legalmente autorizados por la ley.Delegados regionales y seccionales de la Defensoría del Pueblo.Agentes del Ministerio PúblicoJueces de Familia. A falta de todos los anteriores, en el respectivo municipio, esta conciliación podrá ser adelantada por los Personeros y los Jueces Civiles y Promiscuos Municipales. Cabe aclarar que en los lugares del territorio colombiano en donde el Bienestar Familiar no preste sus servicios, así no exista violencia intrafamiliar, podrán acudir a la Comisaria de Familia o Inspección de Policía más cercana al lugar de residencia del niño, niña o adolescente."
https://www.icbf.gov.co/es-valido-que-los-padres-de-comun-acuerdo-definan-quien-tendra-la-custodia-sin-necesidad-de-acudir,1,"Sí, es completamente valido que los padres definan la custodia de forma privada sin acudir a ninguna entidad; sin embargo lo recomendable es que lo hagan por escrito ante una autoridad competente. Es preciso recordar que cualquier decisión que se tome respecto a los hijos debe propender por su bienestar."
https://www.icbf.gov.co/en-colombia-esta-regulada-la-custodia-compartida,1,"No, en Colombia no existe una norma expresa sobre la denominada custodia compartida."
https://www.icbf.gov.co/si-uno-de-los-progenitores-de-un-nino-fue-privadoa-de-la-patria-potestad-se-requiere-de-su-permiso,1,"No se requiere el permiso del padre o madre a qu

##Step 4: Create Vector Store

With our data divided into chunks, we are ready to convert these records into searchable embeddings. Our first step is to separate the content that will be converted from the content that will serve as the metadata surrounding the document:

In [0]:
# convert inputs to pandas dataframe
inputs = chunked_inputs.toPandas()

# extract searchable text elements
text_inputs = inputs['text'].to_list()

# extract metadata
metadata_inputs = (
  inputs
    .drop(['text','num_chunks'], axis=1)
    .to_dict(orient='records')
  )

Next, we will initialize the vector store into which we will load our data.  If you are not familiar with vector stores, these are specialized databases that store text data as embeddings and enable fast searches based on content similarity.  We will be using the [FAISS vector store](https://faiss.ai/) developed by Facebook AI Research. It's fast and lightweight, characteristics that make it ideal for our scenario.

The key to setting up the vector store is to configure it with an embedding model that it will use to convert both the documents and any searchable text to an embedding (vector). You have a wide range of choices available to you as you consider which embedding model to employ.  Some popular models include the [sentence-transformer](https://huggingface.co/models?library=sentence-transformers&sort=downloads) family of models available on the HuggingFace hub as well as the [OpenAI embedding models](https://platform.openai.com/docs/guides/embeddings/what-are-embeddings):

**NOTE** The OpenAI API key used by the OpenAIEmbeddings object is specified in an environment variable set during the earlier `%run` call to get configuration variables.

In [0]:
# Using my own OpenAI API key
api_key_path = '/Workspace/Users/aurelia.ayala@factored.ai/ICBF_QABot/utils/.openai.key'
with open(api_key_path, "r") as f:
    private_oai_key = f.readline()

In [0]:
# identify embedding model that will generate embedding vectors
# Warning: Chunk Size will help a lot in getting this actually working. I set it up to 50, but theoretically larger numbers will make the process much faster.
embeddings = OpenAIEmbeddings(model=config['openai_embedding_model'], chunk_size=50)

In [0]:
# Instantiate vector store object
# Warning: Will output A LOT of RateLimitError messages, but it will eventually work.
vector_store = FAISS.from_texts(
  embedding=embeddings, 
  texts=text_inputs, 
  metadatas=metadata_inputs
  )

So that we make use of our vector store in subsequent notebooks, let's persist it to storage:

In [0]:
vector_store.save_local(folder_path=config['vector_store_path'])
print(f"The vector store was saved in the current location: {config['vector_store_path']}")

The vector store was saved in the current location: /dbfs/tmp/icbf_qabot/vector_store


© 2023 Databricks, Inc. All rights reserved. The source in this notebook is provided subject to the Databricks License. All included or referenced third party libraries are subject to the licenses set forth below.

| library                                | description             | license    | source                                              |
|----------------------------------------|-------------------------|------------|-----------------------------------------------------|
| langchain | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/langchain/ |
| tiktoken | Fast BPE tokeniser for use with OpenAI's models | MIT  |   https://pypi.org/project/tiktoken/ |
| faiss-cpu | Library for efficient similarity search and clustering of dense vectors | MIT  |   https://pypi.org/project/faiss-cpu/ |
| openai | Building applications with LLMs through composability | MIT  |   https://pypi.org/project/openai/ |